## RNN pytorch

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

batch_size = len(sentences)
n_step = 2
n_hidden = 5

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(input)
        target_batch.append(target)

    return torch.LongTensor(input_batch), torch.LongTensor(target_batch)

input_batch, target_batch = make_batch(sentences)


In [ ]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.embedding = nn.Embedding(n_class, n_class)  # Using embedding layer
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
        self.fc = nn.Linear(n_hidden, n_class)  # Fully connected layer

    def forward(self, X):
        X = self.embedding(X)
        X, _ = self.rnn(X)
        X = self.fc(X[:, -1, :])
        return X

model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

predict = model(input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

## LSTM Pytorch

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]

# Word processing
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

# TextLSTM Parameter
batch_size = len(sentences)
n_step = 2
n_hidden = 5

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(input)
        target_batch.append(target)
  
    return torch.LongTensor(input_batch), torch.LongTensor(target_batch)

input_batch, target_batch = make_batch(sentences)

# TextLSTM
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
        self.embedding = nn.Embedding(n_class, n_class)  # Embedding layer
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden)  # LSTM layer
        self.fc = nn.Linear(n_hidden, n_class)  # Fully connected layer

    def forward(self, X):
        X = self.embedding(X)
        X = X.transpose(0, 1)
        outputs, _ = self.lstm(X)
        outputs = self.fc(outputs[-1])
        return outputs

model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training
for epoch in range(500):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

# Prediction
predict = model(input_batch).data.max(1, keepdim=True)[1]
predicted_words = [number_dict[n.item()] for n in predict.squeeze()]
print([sen.split()[:2] for sen in sentences], '->', predicted_words)
